**داتاااااااااااااااااا**

In [1]:
import os, random, shutil
from pathlib import Path
import cv2
import numpy as np

!pip install mtcnn opencv-python

from mtcnn import MTCNN
from google.colab import drive

drive.mount('/content/drive')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 83.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 84.1 MB/s eta 0:00:00
Mounted at /content/drive


In [2]:
RAW_DIR = Path("/content/drive/MyDrive/AI_Project/Final_data_raw/Final_data_raw")

EXP_DIR = Path("/content/drive/MyDrive/AI_Project/experiment")
ALIGNED_DIR = EXP_DIR / "aligned_224"
SPLIT_DIR   = EXP_DIR / "split"

TRAIN_DIR = SPLIT_DIR / "train"
VAL_DIR   = SPLIT_DIR / "val"
TEST_DIR  = SPLIT_DIR / "test"

for d in [ALIGNED_DIR, TRAIN_DIR, VAL_DIR, TEST_DIR]:
    d.mkdir(parents=True, exist_ok=True)

In [3]:
IMG_SIZE = 224
MAX_CLASSES = 30
MAX_IMAGES_PER_CLASS = 200
MIN_IMAGES_AFTER_ALIGNMENT = 120   # مهم جدًا

In [6]:
detector = MTCNN()

# اختيار الأشخاص بشكل ثابت
persons = sorted([p for p in RAW_DIR.iterdir() if p.is_dir()])
persons = persons[:MAX_CLASSES]

print("Target classes:", len(persons))

for person_folder in persons:
    person_out = ALIGNED_DIR / person_folder.name
    person_out.mkdir(parents=True, exist_ok=True)  # 🔴 الحل هنا

    images = list(person_folder.glob("*"))[:MAX_IMAGES_PER_CLASS]

    for img_path in images:
        img = cv2.imread(str(img_path))
        if img is None:
            continue

        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        faces = detector.detect_faces(img_rgb)
        if len(faces) == 0:
            continue

        face = max(faces, key=lambda f: f['box'][2] * f['box'][3])
        x, y, w, h = face['box']
        x, y = max(0, x), max(0, y)

        face_img = img[y:y+h, x:x+w]
        if face_img.size == 0:
            continue

        face_img = cv2.resize(face_img, (IMG_SIZE, IMG_SIZE))
        cv2.imwrite(str(person_out / img_path.name), face_img)

print("✅ Face alignment finished")

Target classes: 30
✅ Face alignment finished


In [7]:
valid_persons = []

for person in sorted(ALIGNED_DIR.iterdir()):
    if not person.is_dir():
        continue

    images = list(person.glob("*"))
    if len(images) >= MIN_IMAGES_AFTER_ALIGNMENT:
        valid_persons.append(person)

print("Valid classes after alignment:", len(valid_persons))

Valid classes after alignment: 30


In [8]:
random.seed(42)

for person in valid_persons:
    images = list(person.glob("*"))
    random.shuffle(images)

    n = len(images)
    n_train = int(0.7 * n)
    n_val   = int(0.1 * n)

    train_imgs = images[:n_train]
    val_imgs   = images[n_train:n_train+n_val]
    test_imgs  = images[n_train+n_val:]

    for split_dir, imgs in zip(
        [TRAIN_DIR, VAL_DIR, TEST_DIR],
        [train_imgs, val_imgs, test_imgs]
    ):
        dest = split_dir / person.name
        dest.mkdir(parents=True, exist_ok=True)

        for img in imgs:
            shutil.copy2(img, dest / img.name)

print("✅ Split 70/20/10 completed")


✅ Split 70/20/10 completed


In [9]:
print("Train classes:", len(list(TRAIN_DIR.iterdir())))
print("Val classes:", len(list(VAL_DIR.iterdir())))
print("Test classes:", len(list(TEST_DIR.iterdir())))

Train classes: 30
Val classes: 30
Test classes: 30
